In [1]:
library("DEP")
library("dplyr")

Warning message:
"package 'DEP' was built under R version 3.5.2"
Warning message in fun(libname, pkgname):
"mzR has been built against a different Rcpp version (1.0.0)
than is installed on your system (1.0.3). This might lead to errors
when loading mzR. If you encounter such issues, please send a report,
including the output of sessionInfo() to the Bioc support forum at 
https://support.bioconductor.org/. For details see also
https://github.com/sneumann/mzR/wiki/mzR-Rcpp-compiler-linker-issue."
Warning message:
"package 'dplyr' was built under R version 3.5.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
type = "LFQ" ; target_path = "AGG_LFQ_DEP_DE.csv"
# type = "iBAQ"; target_path = "AGG_IBAQ_DEP_DE.csv"
# type = "Intensity"; target_path = "AGG_INTENSITY_DEP_DE.csv"

file <- paste0('../../data//',type,'_profiles_agg_cells.csv')
ed_file <-'../../data//profile_agg_experimental_design.csv'

data <- read.csv2(file,sep = ",")
# experimental_design <- read.table(ed_file, header=TRUE, sep="\t", stringsAsFactors = FALSE)
experimental_design <- read.table(ed_file, header=TRUE, sep=";", stringsAsFactors = FALSE)
experimental_design$X <- NULL

In [3]:
# ed_file <-'../../data//profile_agg_experimental_design.csv'
# cells_list <- colnames(data)[grep("._0", colnames(data))]
# label <- cells_list
# condition <- sapply(cells_list,function(x)  strsplit(x,"._0")[[1]][1])
# replicate <- sapply(cells_list,function(x)  strsplit(x,"._0")[[1]][2])

# experimental_design <- data.frame(label,condition,replicate)

In [4]:
data_unique <- make_unique(data,'major_protein_id' ,"Gene.names",delim = " ")

LFQ_columns <- grep("_0", colnames(data_unique))
data_unique[,LFQ_columns] <- apply(data_unique[,LFQ_columns],2,as.numeric)

In [5]:
data_se <- make_se(data_unique, LFQ_columns, experimental_design)

data_filt <- filter_missval(data_se, thr = 0)

In [6]:
data_norm <- normalize_vsn(data_filt)
data_imp <- impute(data_norm, fun = "MinProb", q = 0.01)

Loading required package: imputeLCMD

Warning message:
"package 'imputeLCMD' was built under R version 3.5.3"
Loading required package: tmvtnorm

Warning message:
"package 'tmvtnorm' was built under R version 3.5.3"
Loading required package: mvtnorm

Warning message:
"package 'mvtnorm' was built under R version 3.5.3"
Loading required package: Matrix

Warning message:
"package 'Matrix' was built under R version 3.5.3"
Loading required package: stats4

Loading required package: gmm

Warning message:
"package 'gmm' was built under R version 3.5.3"
Loading required package: sandwich

Warning message:
"package 'sandwich' was built under R version 3.5.3"
Loading required package: norm

Warning message:
"package 'norm' was built under R version 3.5.2"
Loading required package: pcaMethods

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:parallel':


[1] 0.7479245


In [7]:
data_diff_all_contrasts <- test_diff(data_imp,  type = "all")
dep <- add_rejections(data_diff_all_contrasts, alpha = 0.05, lfc = log2(1.5))

data_results <- get_results(dep)

Tested contrasts: B.cells_vs_CD4..effector.memory.T.cell, B.cells_vs_CD4.central.memory.Tcell, B.cells_vs_CD8..effector.memory.T.cell, B.cells_vs_Monocyte, B.cells_vs_Na.ve.CD4.Tcel, B.cells_vs_Na.ve.CD8.Tcel, B.cells_vs_Plasmacytoid.dendritic.cell, CD4..effector.memory.T.cell_vs_CD4.central.memory.Tcell, CD4..effector.memory.T.cell_vs_CD8..effector.memory.T.cell, CD4..effector.memory.T.cell_vs_Monocyte, CD4..effector.memory.T.cell_vs_Na.ve.CD4.Tcel, CD4..effector.memory.T.cell_vs_Na.ve.CD8.Tcel, CD4..effector.memory.T.cell_vs_Plasmacytoid.dendritic.cell, CD4.central.memory.Tcell_vs_CD8..effector.memory.T.cell, CD4.central.memory.Tcell_vs_Monocyte, CD4.central.memory.Tcell_vs_Na.ve.CD4.Tcel, CD4.central.memory.Tcell_vs_Na.ve.CD8.Tcel, CD4.central.memory.Tcell_vs_Plasmacytoid.dendritic.cell, CD8..effector.memory.T.cell_vs_Monocyte, CD8..effector.memory.T.cell_vs_Na.ve.CD4.Tcel, CD8..effector.memory.T.cell_vs_Na.ve.CD8.Tcel, CD8..effector.memory.T.cell_vs_Plasmacytoid.dendritic.cell, Mon

In [9]:
# write.csv2(data_results,target_path)